In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import catboost
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from lightgbm import LGBMClassifier
train=pd.read_csv(r'C:\Users\antip\Downloads\kaggle\train_final.csv')
test=pd.read_csv(r'C:\Users\antip\Downloads\kaggle\test_final.csv')
submition=pd.read_csv(r'C:\Users\antip\Downloads\kaggle\sample_submition.csv')
train_x=train.drop('Onshore/Offshore',axis=1).copy()
train_y=train['Onshore/Offshore'].copy()
train_test=pd.concat([train_x,test],axis=0)
train

,Tectonic regime,Onshore/Offshore,Hydrocarbon type,Reservoir status,Structural setting,Depth,Period,Lithology,Gross,Netpay,Porosity,Permeability
0,COMPRESSION,ONSHORE,OIL,MATURE PRODUCTION,FORELAND,15250.000000,PALEOGENE,SANDSTONE,8000.000000,575.000000,5.000000,0.100000
1,COMPRESSION/EVAPORITE/EXTENSION/INVERSION,OFFSHORE,GAS-CONDENSATE,PLATEAU PRODUCTION,RIFT/SALT,16360.000000,JURASSIC,SANDSTONE,980.000000,490.000000,16.000000,10.000000
2,COMPRESSION/EROSION,ONSHORE,OIL,DECLINING PRODUCTION,FORELAND,4700.000000,PERMIAN,DOLOMITE,65.000000,21.000000,8.000000,0.500000
3,COMPRESSION/EVAPORITE/EXTENSION/GRAVITY/INVERSION,OFFSHORE,OIL,DEVELOPING,FORELAND/RIFT/SALT,8105.345849,CRETACEOUS,SANDSTONE,116.037254,41.595776,14.984042,29.920211
4,COMPRESSION/EROSION,ONSHORE,OIL,NEARLY DEPLETED,FORELAND,7600.000000,CRETACEOUS,SANDSTONE,158.000000,15.000000,9.500000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
350,COMPRESSION/EROSION,ONSHORE-OFFSHORE,OIL,MATURE PRODUCTION,WRENCH/FORELAND,1400.000000,NEOGENE,SANDSTONE,500.000000,120.000000,35.000000,1200.000000
351,COMPRESSION/EROSION,ONSHORE-OFFSHORE,OIL,NEARLY DEPLETED,FORELAND,6550.000000,CRETACEOUS,DOLOMITE,1300.000000,200.000000,11.000000,10.000000
352,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,ONSHORE-OFFSHORE,OIL,NEARLY DEPLETED,WRENCH,2326.000000,NEOGENE,THINLY-BEDDED SANDSTONE,700.000000,375.000000,28.000000,400.000000
353,STRIKE-SLIP/TRANSPRESSION/BASEMENT-I,ONSHORE-OFFSHORE,OIL,NEARLY DEPLETED,WRENCH,1726.000000,NEOGENE,THINLY-BEDDED SANDSTONE,660.000000,152.000000,34.000000,2300.000000


In [26]:
le=LabelEncoder()
train_y=le.fit_transform(train_y)
train_y

array([1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,

In [27]:
train_test=pd.concat([train_test,train_test['Tectonic regime'].str.get_dummies('/').add_prefix('Tectonic_')]
                  ,axis=1).drop('Tectonic regime',axis=1)
train_test=pd.concat([train_test,train_test['Structural setting'].str.get_dummies('/').add_prefix('Structure_')]
                  ,axis=1).drop('Structural setting',axis=1)
train_test=pd.concat([train_test,train_test['Reservoir status'].str.get_dummies().add_prefix('Reservoir_')]
                  ,axis=1).drop('Reservoir status',axis=1)
train_test=pd.concat([train_test,train_test['Period'].str.get_dummies().add_prefix('Period_')]
                  ,axis=1).drop('Period',axis=1)
train_test=pd.concat([train_test,train_test['Lithology'].str.get_dummies().add_prefix('Lithology_')]
                  ,axis=1).drop('Lithology',axis=1)
train_test=pd.concat([train_test,train_test['Hydrocarbon type'].str.get_dummies().add_prefix('Hydrocarbon_')]
                  ,axis=1).drop('Hydrocarbon type',axis=1)

In [20]:
clf=catboost.CatBoostClassifier(verbose=0)
train_2=train_test.iloc[:train_x.shape[0],:]
test_2=train_test.iloc[train_x.shape[0]:,:]
cross_val_score(clf,train_2,train_y).mean()

0.904225352112676

без pca - 0.904225352112676
pca 50 - 0.922870745900194
pca 70 - 0.9209310527243872
pca 40 - 0.9191147945688591
pca 60 - 0.9322694410156938
rfe 60 lightgbm - 0.943519661435373
rfe 50 lightgbm - 0.9397637101040381
rfe 70 lightgbm - 0.9416328689825427

In [24]:
oversample = SMOTE(k_neighbors=3)
train_3, train_y3 = oversample.fit_resample(train_test.iloc[:train_x.shape[0],:], train_y)
pl_pca=Pipeline([('scaler', StandardScaler()), 
          ('pca', PCA(n_components=60)),
          ('catboost', catboost.CatBoostClassifier(verbose=0))])
cross_val_score(pl_pca,train_3,train_y3).mean()

0.9322694410156938

In [34]:
oversample = SMOTE(k_neighbors=3)
train_3, train_y3 = oversample.fit_resample(train_test.iloc[:train_x.shape[0],:], train_y)
pl_rfe=Pipeline([('scaler', StandardScaler()), 
          ('rfe', RFE(LGBMClassifier(),n_features_to_select=70)),
          ('catboost', catboost.CatBoostClassifier(verbose=1))])
cross_val_score(pl_rfe,train_3,train_y3).mean()

Learning rate set to 0.075739
0:	learn: 1.0186404	total: 3.05ms	remaining: 3.05s
1:	learn: 0.9610710	total: 6.17ms	remaining: 3.08s
2:	learn: 0.9120799	total: 8.76ms	remaining: 2.91s
3:	learn: 0.8584264	total: 11.9ms	remaining: 2.96s
4:	learn: 0.8152136	total: 14.5ms	remaining: 2.88s
5:	learn: 0.7809668	total: 17.1ms	remaining: 2.83s
6:	learn: 0.7477251	total: 20.1ms	remaining: 2.85s
7:	learn: 0.7147094	total: 22.7ms	remaining: 2.81s
8:	learn: 0.6860892	total: 25.2ms	remaining: 2.77s
9:	learn: 0.6579007	total: 27.9ms	remaining: 2.76s
10:	learn: 0.6314430	total: 30.5ms	remaining: 2.74s
11:	learn: 0.6080098	total: 33.1ms	remaining: 2.73s
12:	learn: 0.5856456	total: 36.2ms	remaining: 2.75s
13:	learn: 0.5629505	total: 39.3ms	remaining: 2.77s
14:	learn: 0.5420415	total: 42.2ms	remaining: 2.77s
15:	learn: 0.5228491	total: 44.8ms	remaining: 2.76s
16:	learn: 0.5035391	total: 47.5ms	remaining: 2.75s
17:	learn: 0.4868304	total: 50.6ms	remaining: 2.76s
18:	learn: 0.4716985	total: 53.3ms	remaining

210:	learn: 0.0561490	total: 523ms	remaining: 1.96s
211:	learn: 0.0559177	total: 525ms	remaining: 1.95s
212:	learn: 0.0557186	total: 528ms	remaining: 1.95s
213:	learn: 0.0553998	total: 531ms	remaining: 1.95s
214:	learn: 0.0550421	total: 534ms	remaining: 1.95s
215:	learn: 0.0548168	total: 536ms	remaining: 1.95s
216:	learn: 0.0544795	total: 539ms	remaining: 1.94s
217:	learn: 0.0541112	total: 541ms	remaining: 1.94s
218:	learn: 0.0538894	total: 544ms	remaining: 1.94s
219:	learn: 0.0535025	total: 546ms	remaining: 1.94s
220:	learn: 0.0532835	total: 549ms	remaining: 1.93s
221:	learn: 0.0530459	total: 551ms	remaining: 1.93s
222:	learn: 0.0527423	total: 553ms	remaining: 1.93s
223:	learn: 0.0524699	total: 556ms	remaining: 1.92s
224:	learn: 0.0522609	total: 558ms	remaining: 1.92s
225:	learn: 0.0520841	total: 561ms	remaining: 1.92s
226:	learn: 0.0518515	total: 563ms	remaining: 1.92s
227:	learn: 0.0516657	total: 566ms	remaining: 1.91s
228:	learn: 0.0512956	total: 568ms	remaining: 1.91s
229:	learn: 

423:	learn: 0.0243448	total: 1.04s	remaining: 1.42s
424:	learn: 0.0242576	total: 1.05s	remaining: 1.42s
425:	learn: 0.0241906	total: 1.05s	remaining: 1.42s
426:	learn: 0.0241010	total: 1.05s	remaining: 1.41s
427:	learn: 0.0240313	total: 1.06s	remaining: 1.41s
428:	learn: 0.0239810	total: 1.06s	remaining: 1.41s
429:	learn: 0.0238946	total: 1.06s	remaining: 1.41s
430:	learn: 0.0238421	total: 1.07s	remaining: 1.42s
431:	learn: 0.0237826	total: 1.07s	remaining: 1.41s
432:	learn: 0.0237193	total: 1.08s	remaining: 1.41s
433:	learn: 0.0236864	total: 1.08s	remaining: 1.41s
434:	learn: 0.0236140	total: 1.08s	remaining: 1.41s
435:	learn: 0.0235623	total: 1.08s	remaining: 1.4s
436:	learn: 0.0235012	total: 1.09s	remaining: 1.4s
437:	learn: 0.0234739	total: 1.09s	remaining: 1.4s
438:	learn: 0.0233954	total: 1.09s	remaining: 1.4s
439:	learn: 0.0233119	total: 1.09s	remaining: 1.39s
440:	learn: 0.0232286	total: 1.1s	remaining: 1.39s
441:	learn: 0.0231217	total: 1.1s	remaining: 1.39s
442:	learn: 0.0230

634:	learn: 0.0148636	total: 1.57s	remaining: 902ms
635:	learn: 0.0148345	total: 1.57s	remaining: 900ms
636:	learn: 0.0148138	total: 1.57s	remaining: 897ms
637:	learn: 0.0147851	total: 1.58s	remaining: 895ms
638:	learn: 0.0147682	total: 1.58s	remaining: 893ms
639:	learn: 0.0147264	total: 1.58s	remaining: 890ms
640:	learn: 0.0147002	total: 1.58s	remaining: 888ms
641:	learn: 0.0146683	total: 1.59s	remaining: 885ms
642:	learn: 0.0146540	total: 1.59s	remaining: 883ms
643:	learn: 0.0146224	total: 1.59s	remaining: 880ms
644:	learn: 0.0145845	total: 1.59s	remaining: 878ms
645:	learn: 0.0145623	total: 1.6s	remaining: 875ms
646:	learn: 0.0145264	total: 1.6s	remaining: 873ms
647:	learn: 0.0144932	total: 1.6s	remaining: 870ms
648:	learn: 0.0144660	total: 1.6s	remaining: 868ms
649:	learn: 0.0144471	total: 1.61s	remaining: 865ms
650:	learn: 0.0144198	total: 1.61s	remaining: 863ms
651:	learn: 0.0143958	total: 1.61s	remaining: 861ms
652:	learn: 0.0143560	total: 1.61s	remaining: 858ms
653:	learn: 0.01

843:	learn: 0.0104497	total: 2.09s	remaining: 386ms
844:	learn: 0.0104384	total: 2.09s	remaining: 384ms
845:	learn: 0.0104218	total: 2.1s	remaining: 382ms
846:	learn: 0.0104164	total: 2.1s	remaining: 379ms
847:	learn: 0.0104052	total: 2.1s	remaining: 377ms
848:	learn: 0.0103888	total: 2.1s	remaining: 374ms
849:	learn: 0.0103712	total: 2.11s	remaining: 372ms
850:	learn: 0.0103544	total: 2.11s	remaining: 369ms
851:	learn: 0.0103397	total: 2.11s	remaining: 367ms
852:	learn: 0.0103268	total: 2.11s	remaining: 364ms
853:	learn: 0.0103151	total: 2.12s	remaining: 362ms
854:	learn: 0.0103013	total: 2.12s	remaining: 359ms
855:	learn: 0.0102864	total: 2.12s	remaining: 357ms
856:	learn: 0.0102731	total: 2.12s	remaining: 354ms
857:	learn: 0.0102549	total: 2.13s	remaining: 352ms
858:	learn: 0.0102367	total: 2.13s	remaining: 349ms
859:	learn: 0.0102193	total: 2.13s	remaining: 347ms
860:	learn: 0.0102057	total: 2.13s	remaining: 344ms
861:	learn: 0.0101961	total: 2.13s	remaining: 342ms
862:	learn: 0.01

...

269:	learn: 0.0396240	total: 701ms	remaining: 1.89s
270:	learn: 0.0394666	total: 704ms	remaining: 1.89s
271:	learn: 0.0392674	total: 707ms	remaining: 1.89s
272:	learn: 0.0391341	total: 710ms	remaining: 1.89s
273:	learn: 0.0389588	total: 713ms	remaining: 1.89s
274:	learn: 0.0387641	total: 715ms	remaining: 1.89s
275:	learn: 0.0386100	total: 718ms	remaining: 1.88s
276:	learn: 0.0384443	total: 721ms	remaining: 1.88s
277:	learn: 0.0382784	total: 723ms	remaining: 1.88s
278:	learn: 0.0381694	total: 725ms	remaining: 1.87s
279:	learn: 0.0379763	total: 728ms	remaining: 1.87s
280:	learn: 0.0378448	total: 730ms	remaining: 1.87s
281:	learn: 0.0376264	total: 733ms	remaining: 1.87s
282:	learn: 0.0375540	total: 735ms	remaining: 1.86s
283:	learn: 0.0374373	total: 738ms	remaining: 1.86s
284:	learn: 0.0372755	total: 741ms	remaining: 1.86s
285:	learn: 0.0370736	total: 743ms	remaining: 1.85s
286:	learn: 0.0368553	total: 746ms	remaining: 1.85s
287:	learn: 0.0366501	total: 748ms	remaining: 1.85s
288:	learn: 

471:	learn: 0.0203051	total: 1.22s	remaining: 1.37s
472:	learn: 0.0202501	total: 1.22s	remaining: 1.36s
473:	learn: 0.0201666	total: 1.23s	remaining: 1.36s
474:	learn: 0.0201314	total: 1.23s	remaining: 1.36s
475:	learn: 0.0200897	total: 1.23s	remaining: 1.36s
476:	learn: 0.0200434	total: 1.24s	remaining: 1.35s
477:	learn: 0.0200077	total: 1.24s	remaining: 1.35s
478:	learn: 0.0199219	total: 1.24s	remaining: 1.35s
479:	learn: 0.0198851	total: 1.24s	remaining: 1.34s
480:	learn: 0.0198477	total: 1.24s	remaining: 1.34s
481:	learn: 0.0197970	total: 1.25s	remaining: 1.34s
482:	learn: 0.0197471	total: 1.25s	remaining: 1.34s
483:	learn: 0.0196790	total: 1.25s	remaining: 1.33s
484:	learn: 0.0196108	total: 1.25s	remaining: 1.33s
485:	learn: 0.0195398	total: 1.26s	remaining: 1.33s
486:	learn: 0.0194932	total: 1.26s	remaining: 1.33s
487:	learn: 0.0194428	total: 1.26s	remaining: 1.32s
488:	learn: 0.0193970	total: 1.26s	remaining: 1.32s
489:	learn: 0.0193622	total: 1.27s	remaining: 1.32s
490:	learn: 

678:	learn: 0.0128922	total: 1.74s	remaining: 825ms
679:	learn: 0.0128695	total: 1.75s	remaining: 823ms
680:	learn: 0.0128464	total: 1.75s	remaining: 820ms
681:	learn: 0.0128188	total: 1.75s	remaining: 817ms
682:	learn: 0.0127964	total: 1.75s	remaining: 815ms
683:	learn: 0.0127615	total: 1.76s	remaining: 812ms
684:	learn: 0.0127354	total: 1.76s	remaining: 810ms
685:	learn: 0.0127098	total: 1.76s	remaining: 808ms
686:	learn: 0.0126950	total: 1.77s	remaining: 805ms
687:	learn: 0.0126646	total: 1.77s	remaining: 802ms
688:	learn: 0.0126476	total: 1.77s	remaining: 800ms
689:	learn: 0.0126301	total: 1.77s	remaining: 797ms
690:	learn: 0.0125957	total: 1.78s	remaining: 795ms
691:	learn: 0.0125697	total: 1.78s	remaining: 792ms
692:	learn: 0.0125549	total: 1.78s	remaining: 790ms
693:	learn: 0.0125105	total: 1.78s	remaining: 787ms
694:	learn: 0.0124818	total: 1.79s	remaining: 785ms
695:	learn: 0.0124474	total: 1.79s	remaining: 782ms
696:	learn: 0.0124197	total: 1.79s	remaining: 780ms
697:	learn: 

872:	learn: 0.0094344	total: 2.27s	remaining: 330ms
873:	learn: 0.0094257	total: 2.27s	remaining: 328ms
874:	learn: 0.0094177	total: 2.28s	remaining: 325ms
875:	learn: 0.0094055	total: 2.28s	remaining: 323ms
876:	learn: 0.0093965	total: 2.28s	remaining: 320ms
877:	learn: 0.0093848	total: 2.29s	remaining: 318ms
878:	learn: 0.0093727	total: 2.29s	remaining: 315ms
879:	learn: 0.0093583	total: 2.29s	remaining: 312ms
880:	learn: 0.0093490	total: 2.29s	remaining: 310ms
881:	learn: 0.0093390	total: 2.29s	remaining: 307ms
882:	learn: 0.0093234	total: 2.3s	remaining: 305ms
883:	learn: 0.0093107	total: 2.3s	remaining: 302ms
884:	learn: 0.0093046	total: 2.3s	remaining: 299ms
885:	learn: 0.0092935	total: 2.31s	remaining: 297ms
886:	learn: 0.0092786	total: 2.31s	remaining: 294ms
887:	learn: 0.0092695	total: 2.31s	remaining: 291ms
888:	learn: 0.0092613	total: 2.31s	remaining: 289ms
889:	learn: 0.0092442	total: 2.32s	remaining: 286ms
890:	learn: 0.0092339	total: 2.32s	remaining: 284ms
891:	learn: 0.0

0.9416328689825427

In [35]:
pl_rfe=Pipeline([('scaler', StandardScaler()), 
          ('rfe', RFE(LGBMClassifier(),n_features_to_select=60)),
          ('catboost', catboost.CatBoostClassifier(verbose=1))])
pl_rfe.fit(train_3, train_y3)

Learning rate set to 0.076613
0:	learn: 1.0229551	total: 2.81ms	remaining: 2.8s
1:	learn: 0.9533475	total: 5.59ms	remaining: 2.79s
2:	learn: 0.8932942	total: 8.74ms	remaining: 2.9s
3:	learn: 0.8420889	total: 11.6ms	remaining: 2.88s
4:	learn: 0.7989303	total: 14.5ms	remaining: 2.89s
5:	learn: 0.7635925	total: 17.2ms	remaining: 2.84s
6:	learn: 0.7245476	total: 19.8ms	remaining: 2.81s
7:	learn: 0.6892502	total: 22.3ms	remaining: 2.76s
8:	learn: 0.6569108	total: 24.9ms	remaining: 2.74s
9:	learn: 0.6266097	total: 27.5ms	remaining: 2.72s
10:	learn: 0.6018419	total: 30ms	remaining: 2.7s
11:	learn: 0.5767448	total: 32.5ms	remaining: 2.68s
12:	learn: 0.5553239	total: 35.4ms	remaining: 2.69s
13:	learn: 0.5345950	total: 38ms	remaining: 2.67s
14:	learn: 0.5161187	total: 40.7ms	remaining: 2.67s
15:	learn: 0.5007581	total: 43.4ms	remaining: 2.67s
16:	learn: 0.4840391	total: 46.1ms	remaining: 2.67s
17:	learn: 0.4660345	total: 48.7ms	remaining: 2.66s
18:	learn: 0.4512692	total: 51.3ms	remaining: 2.65s

215:	learn: 0.0528685	total: 523ms	remaining: 1.9s
216:	learn: 0.0525748	total: 526ms	remaining: 1.9s
217:	learn: 0.0521965	total: 528ms	remaining: 1.89s
218:	learn: 0.0519455	total: 531ms	remaining: 1.89s
219:	learn: 0.0517597	total: 533ms	remaining: 1.89s
220:	learn: 0.0516224	total: 536ms	remaining: 1.89s
221:	learn: 0.0513836	total: 539ms	remaining: 1.89s
222:	learn: 0.0511167	total: 541ms	remaining: 1.89s
223:	learn: 0.0508553	total: 544ms	remaining: 1.88s
224:	learn: 0.0506243	total: 546ms	remaining: 1.88s
225:	learn: 0.0504446	total: 549ms	remaining: 1.88s
226:	learn: 0.0502290	total: 551ms	remaining: 1.88s
227:	learn: 0.0499730	total: 553ms	remaining: 1.87s
228:	learn: 0.0496645	total: 556ms	remaining: 1.87s
229:	learn: 0.0494291	total: 558ms	remaining: 1.87s
230:	learn: 0.0491961	total: 560ms	remaining: 1.86s
231:	learn: 0.0489874	total: 563ms	remaining: 1.86s
232:	learn: 0.0487160	total: 565ms	remaining: 1.86s
233:	learn: 0.0485074	total: 567ms	remaining: 1.86s
234:	learn: 0.

436:	learn: 0.0232135	total: 1.05s	remaining: 1.35s
437:	learn: 0.0231211	total: 1.05s	remaining: 1.35s
438:	learn: 0.0230365	total: 1.05s	remaining: 1.34s
439:	learn: 0.0229395	total: 1.05s	remaining: 1.34s
440:	learn: 0.0228728	total: 1.06s	remaining: 1.34s
441:	learn: 0.0228392	total: 1.06s	remaining: 1.34s
442:	learn: 0.0227713	total: 1.06s	remaining: 1.33s
443:	learn: 0.0226959	total: 1.06s	remaining: 1.33s
444:	learn: 0.0226399	total: 1.07s	remaining: 1.33s
445:	learn: 0.0225690	total: 1.07s	remaining: 1.33s
446:	learn: 0.0224885	total: 1.07s	remaining: 1.32s
447:	learn: 0.0224152	total: 1.07s	remaining: 1.32s
448:	learn: 0.0223796	total: 1.08s	remaining: 1.32s
449:	learn: 0.0223223	total: 1.08s	remaining: 1.32s
450:	learn: 0.0222701	total: 1.08s	remaining: 1.31s
451:	learn: 0.0222143	total: 1.08s	remaining: 1.31s
452:	learn: 0.0221754	total: 1.08s	remaining: 1.31s
453:	learn: 0.0221014	total: 1.09s	remaining: 1.31s
454:	learn: 0.0220453	total: 1.09s	remaining: 1.3s
455:	learn: 0

655:	learn: 0.0140394	total: 1.57s	remaining: 825ms
656:	learn: 0.0140021	total: 1.57s	remaining: 822ms
657:	learn: 0.0139720	total: 1.58s	remaining: 820ms
658:	learn: 0.0139425	total: 1.58s	remaining: 817ms
659:	learn: 0.0139300	total: 1.58s	remaining: 815ms
660:	learn: 0.0139049	total: 1.58s	remaining: 813ms
661:	learn: 0.0138739	total: 1.59s	remaining: 811ms
662:	learn: 0.0138512	total: 1.59s	remaining: 808ms
663:	learn: 0.0138288	total: 1.59s	remaining: 806ms
664:	learn: 0.0137981	total: 1.59s	remaining: 803ms
665:	learn: 0.0137530	total: 1.6s	remaining: 801ms
666:	learn: 0.0137095	total: 1.6s	remaining: 799ms
667:	learn: 0.0137019	total: 1.6s	remaining: 796ms
668:	learn: 0.0136734	total: 1.6s	remaining: 794ms
669:	learn: 0.0136500	total: 1.61s	remaining: 791ms
670:	learn: 0.0136184	total: 1.61s	remaining: 789ms
671:	learn: 0.0135890	total: 1.61s	remaining: 786ms
672:	learn: 0.0135764	total: 1.61s	remaining: 784ms
673:	learn: 0.0135619	total: 1.61s	remaining: 782ms
674:	learn: 0.01

872:	learn: 0.0097735	total: 2.1s	remaining: 305ms
873:	learn: 0.0097555	total: 2.1s	remaining: 303ms
874:	learn: 0.0097419	total: 2.1s	remaining: 300ms
875:	learn: 0.0097269	total: 2.1s	remaining: 298ms
876:	learn: 0.0097115	total: 2.11s	remaining: 296ms
877:	learn: 0.0096975	total: 2.11s	remaining: 293ms
878:	learn: 0.0096835	total: 2.11s	remaining: 291ms
879:	learn: 0.0096753	total: 2.11s	remaining: 288ms
880:	learn: 0.0096599	total: 2.12s	remaining: 286ms
881:	learn: 0.0096492	total: 2.12s	remaining: 284ms
882:	learn: 0.0096382	total: 2.12s	remaining: 281ms
883:	learn: 0.0096212	total: 2.12s	remaining: 279ms
884:	learn: 0.0096016	total: 2.13s	remaining: 276ms
885:	learn: 0.0095843	total: 2.13s	remaining: 274ms
886:	learn: 0.0095680	total: 2.13s	remaining: 272ms
887:	learn: 0.0095550	total: 2.13s	remaining: 269ms
888:	learn: 0.0095407	total: 2.14s	remaining: 267ms
889:	learn: 0.0095242	total: 2.14s	remaining: 264ms
890:	learn: 0.0095138	total: 2.14s	remaining: 262ms
891:	learn: 0.00

Pipeline(steps=[('scaler', StandardScaler()),
                ('rfe',
                 RFE(estimator=LGBMClassifier(), n_features_to_select=60)),
                ('catboost',
                 <catboost.core.CatBoostClassifier object at 0x000001B878BCAA00>)])

In [56]:
df=pd.DataFrame(le.inverse_transform(pl_rfe.predict(train_test.iloc[train_x.shape[0]:,:])),columns=['Onshore/Offshore']) 
df.index.rename('Index',inplace=True)
df.to_csv(r'C:\Users\antip\Desktop\a.csv')

c:\users\antip\appdata\local\programs\python\python38\lib\site-packages\sklearn\preprocessing\_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
